# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [ ]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [27]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", "Lat", geo=True, tiles="OSM", c='City', size='Humidity', alpha=0.7, hover_cols=['City', 'Country', 'Hotel Name', 'Humidity']
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [26]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df[
    (city_data_df['Max Temp'] > 15) & 
    (city_data_df['Cloudiness'] > 50) &    # Cloudiness greater than 50%
    (city_data_df['Wind Speed'] > 5)       # Humidity equal to 0
]

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
print(filtered_cities_df.head())

    City_ID               City      Lat       Lng  Max Temp  Humidity  \
23       23            rikitea -23.1203 -134.9692     20.41        65   
34       34           naberera  -4.2000   36.9333     21.12        54   
41       41  sao joao da barra -21.6403  -41.0511     29.05        75   
55       55            cayenne   4.9333  -52.3333     32.02        58   
68       68       port lincoln -34.7333  135.8667     15.02        88   

    Cloudiness  Wind Speed Country        Date  
23          77        9.23      PF  1666108244  
34          94        6.38      TZ  1666108252  
41          60        7.15      BR  1666108256  
55          75        5.14      GF  1666108014  
68          83        5.20      AU  1666108273  


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
print(hotel_df.head())

          City Country      Lat       Lng  Humidity Hotel Name
0         faya      SA  18.3851   42.4509        35           
1      farsund      NO  58.0948    6.8047       100           
2  new norfolk      AU -42.7826  147.0587        58           
3    jamestown      US  42.0970  -79.2353        77           
4      lanzhou      CN  36.0564  103.7922        48           


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat, lng = row['Lat'], row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
faya - nearest hotel: No hotel found
farsund - nearest hotel: Rederiet Hotell
new norfolk - nearest hotel: Woodbridge on the Derwent
jamestown - nearest hotel: DoubleTree Jamestown
lanzhou - nearest hotel: 西园宾馆
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
albany - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
sisimiut - nearest hotel: Hotel SØMA
saint-pierre - nearest hotel: Tropic Hotel
colorado - nearest hotel: Servilha Park Hotel
nikolskoye - nearest hotel: No hotel found
trairi - nearest hotel: No hotel found
dobryanka - nearest hotel: Уральская Венеция
qaanaaq - nearest hotel: Hotel Qaanaaq
mataura - nearest hotel: No hotel found
khatanga - nearest hotel: Хатанга
tasiilaq - nearest hotel: Angmagssalik
tiksi - nearest hotel: Арктика
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
avarua - nearest hotel: Paradise Inn
nuevo laredo - nearest hotel: Best Western Plus
guiratinga - nearest hotel: No hotel found
r

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
city_data_df = city_data_df.drop(columns=['Hotel Name'], errors='ignore')
city_data_df = city_data_df.merge(hotel_df[['City', 'Hotel Name']], on='City', how='left')
print(city_data_df[['City', 'Hotel Name']].head())

          City                 Hotel Name
0         faya             No hotel found
1      farsund            Rederiet Hotell
2  new norfolk  Woodbridge on the Derwent
3    jamestown       DoubleTree Jamestown
4      lanzhou                       西园宾馆


In [20]:
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", "Lat", geo=True, tiles="OSM", c='City', size='Humidity', alpha=0.7, hover_cols=['City', 'Country', 'Hotel Name', 'Humidity']
)
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)